<a href="https://www.kaggle.com/code/marydata/ohada-llm?scriptVersionId=220070680" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# This is a starter notebook to help you start with the hackathon

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ohada-llm/Data/Eval.csv
/kaggle/input/ohada-llm/Data/Test.csv
/kaggle/input/ohada-llm/Data/starter_notebook_Indabax_Cameroon.ipynb
/kaggle/input/ohada-llm/Data/data_pdfs_indabax_cameroon/AU_cooperatives.pdf
/kaggle/input/ohada-llm/Data/data_pdfs_indabax_cameroon/AU_suretes.pdf
/kaggle/input/ohada-llm/Data/data_pdfs_indabax_cameroon/AU_transport.pdf
/kaggle/input/ohada-llm/Data/data_pdfs_indabax_cameroon/AU_collectives.pdf
/kaggle/input/ohada-llm/Data/data_pdfs_indabax_cameroon/AU_ste_com.pdf
/kaggle/input/ohada-llm/Data/data_pdfs_indabax_cameroon/AU_arbitrage.pdf
/kaggle/input/ohada-llm/Data/data_pdfs_indabax_cameroon/AU_arbitrage(1).pdf
/kaggle/input/ohada-llm/Data/data_pdfs_indabax_cameroon/AU_execution.pdf
/kaggle/input/ohada-llm/Data/data_pdfs_indabax_cameroon/AU_DCG.pdf
/kaggle/input/ohada-llm/Data/data_pdfs_indabax_cameroon/AU_mediation.pdf
/kaggle/input/ohada-llm/Data/data_pdfs_indabax_cameroon/AU_comptable.pdf
/kaggle/input/ohada-llm/Data/data_json_indabax_cameroo

# Large Language Model Challenge on [OHADA](https://www.ohada.org/) Law


**[Inspired from the blog post by ai geek (wishesh)](https://levelup.gitconnected.com/building-a-private-ai-chatbot-2c071f6715ad)**

In [2]:
pip install datasets 

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers peft  accelerate bitsandbytes safetensors sentencepiece streamlit chromadb langchain sentence-transformers gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
pip install -U langchain-community

  Using cached pydantic_core-2.27.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.6 MB/s eta 0:00:00
Using cached pydantic_core-2.27.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.18.1
    Uninstalling pydantic_core-2.18.1:
      Successfully uninstalled pydantic_core-2.18.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.3 requires typer<0.10.0,>=0.3.0, but you have typer 0.15.1 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.15.1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import TextLoader
from langchain.llms import HuggingFacePipeline
from huggingface_hub import notebook_login
from langchain import HuggingFacePipeline
from langchain.vectorstores import Chroma
from datasets import load_dataset
import gradio as gr
import chromadb
import locale
import torch
import os

2025-01-31 12:10:37.072740: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-31 12:10:37.072902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-31 12:10:37.229139: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
# fixing unicode error in google colab
locale.getpreferredencoding = lambda: "UTF-8"

In [7]:
#Make sure to create a huggingface account and generate your token to login with here.
notebook_login()

# Load Model

In [8]:
# specify model huggingface mode name
model_name = "Trelis/Llama-2-7b-chat-hf-sharded-bf16"

###### other models:
# "Trelis/Llama-2-7b-chat-hf-sharded-bf16"
# "bn22/Mistral-7B-Instruct-v0.1-sharded"
# "mistralai/Mistral-7B-Instruct-v0.2"
# "HuggingFaceH4/zephyr-7b-beta"
# "anakin87/zephyr-7b-alpha-sharded"
# function for loading 4-bit quantized model

def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        #load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config
    )
    return model

# fucntion for initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

In [9]:
# load model
model = load_quantized_model(model_name)

# initialize tokenizer
tokenizer = initialize_tokenizer(model_name)

# specify stop token ids
stop_token_ids = [0]

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

...

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

# Create Embeddings

#### Data have two formats: pdfs and jsons, here we will show you how to load the json files from hugginface or from a folder located in your drive. You have to load also the pdfs files and merge them with the json files to create your RAG.

###### Load data

In [10]:
# load data from hugging face

dataset = load_dataset("uriel/Maathis_Ohada_dataset", "default")

Resolving data files:   0%|          | 0/1184 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'content', 'details'],
        num_rows: 1184
    })
})

In [12]:
len(dataset['train'])

1184

In [13]:
dataset['train'][0]['content']

'COUR COMMUNE DE JUSTICE ET ARBITRAGEDemande d\'avis n0 001/99Président du Tribunal Judiciairede Première Instance de Libreville(République Gabonaise)AVIS N° OO1/99/JN Séance du 7 juillet 1999\nLa Cour Commune de Justice et d\'Arbitrage, réunie en formation plénière à son siège le 7 juillet 1999,Vu le traité relatif à l\'harmonisation du Droit des Affaires en Afrique, notamment en ses articles 13 et 14 ; Vu le Règlement de procédure de la Cour Commune de Justice et d\'Arbitrage, notamment en ses articles 9,53,56,57 et 58 ;Vu la demande d\'avis formulée le 13 janvier 1999 par le Président du Tribunal judiciaire de première instance de Libreville (République Gabonaise), reçue le 1er mars 1999, dans une instance opposant la société X et Monsieur A à la société X et Monsieur B, et ainsi libellée :«...Donner son avis consultatif sur :1) Le régime juridique des nullités institué par l\'Acte uniforme (portant organisation des procédures simplifiées de recouvrement et des voies d\'exécution), 

In [14]:
texts = []  # Initialize texts as an empty list to store file contents

for number in range(len(dataset['train'])):
    # if file.endswith('.json'):
    #     file_path = os.path.join(folder_path, file)
    #     with open(file_path, 'r', encoding='utf-8') as open_file:
        content = dataset['train'][number]['content']
        texts.append(content)

print(len(texts))


# load PDF files from a directory
loader = PyPDFDirectoryLoader("/kaggle/input/ohada-llm/Data/data_pdfs_indabax_cameroon/")
data = loader.load()
# print the loaded data, which is a list of tuples (file name, text extracted from the PDF)
print(data)

1184
[Document(metadata={'source': '/kaggle/input/ohada-llm/Data/data_pdfs_indabax_cameroon/AU_cooperatives.pdf', 'page': 0, 'page_label': '1'}, page_content='www.Droit-Afrique.com  OHADA \n \nActe uniforme OHADA relatif au dr oit des sociétés coopératives 1OHADA \nDroit des sociétés coopératives \nActe uniforme OHADA du 15 décembre 2010 \n \n[NB - Acte uniforme OHADA du 15 décembre 2010 relatif au droit des sociétés coopératives ] \n \nSommaire \n \nPARTIE 1 - D ISPOSITIONS GÉNÉRALES SUR LA SOCIÉTÉ COOPÉRATIVE .............................. 4  \nTitre 1 - Constitution de la société coopérative ................................................................ 4  \nChapitre 1 - Définition de la société coopérative et princi pes coopératifs..................... 4 \nChapitre 2 - La qualité d’associé.................................................................................... 5 \nSection 1 - Dispositions générales ......................................................................

###### Or

In [15]:
# # mount google drive and specify folder path
# from google.colab import drive
# drive.mount('/content/drive')
# folder_path = '/content/drive/MyDrive/Garoua/Hackathon/data/' #change to data path in your drive
# texts = []  # Initialize texts as an empty list to store file contents

# for file in os.listdir(folder_path):
#     if file.endswith('.json'):
#         file_path = os.path.join(folder_path, file)
#         with open(file_path, 'r', encoding='utf-8') as open_file:
#             content = open_file.read()
#             texts.append(content)

# print(len(texts))


In [16]:
class Document:
    def __init__(self, content, metadata=None):
        self.page_content = content
        # Initialize metadata as an empty dict if none provided
        self.metadata = metadata if metadata is not None else {}

In [17]:
documents = [Document(content=text) for text in texts]
# split the documents in small chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separators=["\n\n", "\n", ".", " ", ""],) #Change the chunk_size and chunk_overlap as needed
all_splits = text_splitter.split_documents(documents)



# split the extracted data into text chunks using the text_splitter, which splits the text based on the specified number of characters and overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separators=["\n\n", "\n", ".", " ", ""],)
text_chunks = text_splitter.split_documents(data)
# print the number of chunks obtained
len(text_chunks)

2146

In [18]:
# specify embedding model (using huggingface sentence transformer)
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)

/tmp/ipykernel_24/3984801029.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs=model_kwargs)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
folder_path2 = '/kaggle/working/Vector_db/' #change to path in your drive where you want to store the vector db
# embed document chunks
# embeddings are stored in a database (presumably on disk) specified by persist_directory
# ensure that folder_path2 is correctly defined and points to a valid directory where you have write permissions
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory= folder_path2 + "chroma_db")
retrieverts = vectordb.as_retriever()
#How to load
#vectordbts = Chroma(persist_directory= folder_path2 + "chroma_db", embedding_function=embeddings)


vectordb = Chroma.from_documents(documents=text_chunks, embedding=embeddings, persist_directory= folder_path2 + "chromapdfs_db")
retrievertspdf = vectordb.as_retriever(documents=text_chunks)


# Create Pipeline

In [20]:
text_generation_pipeline = pipeline("text-generation", model=model,tokenizer=tokenizer,  max_length=4096, )

# specify the llm
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

def create_conversation(query: str, chat_history: list) -> tuple:
    try:
        # Object to hold the chat history
        memory = ConversationBufferMemory(
            memory_key='chat_history',
            return_messages=False
        )
        # chain setup
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retrieverts,
            memory=memory,
            get_chat_history=lambda h: h,
        )
        
        qa_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=retrievertspdf,
            memory=memory,
            get_chat_history=lambda h: h,
        )
        
        # get llm response with rag and history
        result = qa_chain({'question': query, 'chat_history': chat_history})
        # add the query and response to chat history
        chat_history.append((query, result['answer']))
        return '', chat_history

    # catch exceptions, log them into the chat history
    except Exception as e:
        chat_history.append((query, e))
        return '', chat_history

Device set to use cuda:0
/tmp/ipykernel_24/2924533976.py:4: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [21]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

test_query = "Qu'est-ce que l'OHADA ?"
test_history = []
create_conversation(test_query, test_history)

/tmp/ipykernel_24/2924533976.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
/tmp/ipykernel_24/2924533976.py:29: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({'question': query, 'chat_history': chat_history})
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` 

('',
 [("Qu'est-ce que l'OHADA ?",
   "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nwww.Droit-Afrique.com  OHADA \n \nActe uniforme OHADA portant sur le droit commercial général 3 \nLIVRE 8 - V ENTE COMMERCIALE ........................................................................................ 55  \nTitre 1 - Champ d’application et dispositions générales................................................ 55  \nChapitre 1 - Champ d’application ................................................................................ 55 \nChapitre 2 - Dispositions générales.............................................................................. 55 \nTitre 2 - Formation du contrat........................................................................................ 56  \nTitre 3 - Obligations des parties...................................................................

#### As you can see, the answer provided by the model is in english, try to get the answer in French

# Prompt-Engineering

In [22]:
test_query = "What is OHADA ?"
test_prompt = f"""<s>[INST] {test_query}  [/INST] """
test_history = []
create_conversation(test_prompt, test_history)

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


('',
 [('<s>[INST] What is OHADA ?  [/INST] ',
   "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nwww.Droit-Afrique.com  OHADA \n \nActe uniforme OHADA portant sur le droit commercial général 66dix jours à compter de la date de sa publi cation au Journal officiel de l’OHADA conformé-\nment à l’article 9 du Traité relatif à l’harmoni sation du droit des affaires en Afrique, signé à \nport Louis le 17 octobre 1993, tel que révisé à Québec le 17 octobre 2008.\n\nwww.Droit-Afrique.com  OHADA \n \nActe uniforme OHADA portant organisation des sûretés 4Chapitre 2 - Agent des sûretés \nArt.5.-  Toute sûreté ou autre garantie de l’exécution d’une obligation peut être constituée, \ninscrite, gérée et réalisée par une institution fi nancière ou un établissement de crédit, national \nou étranger, agissant, en son nom et en qualité d’ agent des sûretés, au profit des créanciers de 

# Gradio UI

In [23]:
# build gradio ui
with gr.Blocks() as demo:

    chatbot = gr.Chatbot(label='Chat with your Ohada dataset ;)')
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    torch.backends.cuda.enable_mem_efficient_sdp(False)
    torch.backends.cuda.enable_flash_sdp(False)

    msg.submit(create_conversation, [msg, chatbot], [msg, chatbot])

demo.launch()

/opt/conda/lib/python3.10/site-packages/gradio/components/chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://02b439e48a86306c88.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Good Luck for the Competition !!!!

### Be creative!!!